In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# 윈도우 기본 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [3]:
df = pd.read_excel('C:\\Users\\User\\Documents\\OneDrive - SangMyung University\\모멘텀\\오합지존 프로젝트\\최종 데이터셋.xlsx')

In [4]:
import pandas as pd

def classify_delivery(group):
    vals = group['배달매출금액 비율'].tolist()
    valid_vals = [v for v in vals if v != -999999.9]

    # (1) 모든 값이 조사X라면
    if len(valid_vals) == 0:
        return pd.Series({'평균배달비율': None, '배달유형': '조사X'})

    # (2) 평균 계산
    mean_val = sum(valid_vals) / len(valid_vals)

    # (3) 평균 기준 분류
    if mean_val < 20:
        type_ = '홀전문점'
    elif mean_val <= 80:
        type_ = '병행형'
    else:
        type_ = '배달전문점'

    return pd.Series({'평균배달비율': round(mean_val, 2), '배달유형': type_})

# 🔹 가맹점별 평균 + 유형 계산
result = (
    df.groupby('가맹점구분번호', group_keys=False)
      .apply(classify_delivery)
      .reset_index()
)

# 🔹 원본에 병합
df = df.merge(result, on='가맹점구분번호', how='left')


In [5]:
# 가맹점 단위로 중복 제거
unique_df = (
    df[['가맹점구분번호', '평균배달비율', '배달유형', '업종', '상권']]
    .drop_duplicates(subset=['가맹점구분번호'])
    .reset_index(drop=True)
)

unique_df.to_excel('배달유형_가맹점단위.xlsx', index=False)

In [6]:
result = unique_df.drop_duplicates(subset=['가맹점구분번호', '배달유형'])
result['배달유형'].value_counts()

배달유형
조사X      2483
홀전문점      915
병행형       490
배달전문점     155
Name: count, dtype: int64

In [7]:
# 배달유형별 업종 개수 테이블
업종_분포 = (
    result.groupby(['업종','배달유형'])
      .size()
      .reset_index(name='가맹점수')
      .sort_values(['배달유형', '가맹점수'], ascending=[True, False])
)

# 배달유형별 비율 계산
업종_비율 = (
    업종_분포
    .groupby('배달유형')
    .apply(lambda x: x.assign(비율=round(x['가맹점수'] / x['가맹점수'].sum() * 100, 2)))
    .reset_index(drop=True)
)

업종_비율

,업종,배달유형,가맹점수,비율
0,백반/가정식,배달전문점,25,16.13
1,치킨,배달전문점,21,13.55
2,한식-단품요리일반,배달전문점,21,13.55
3,한식-육류/고기,배달전문점,20,12.90
4,양식,배달전문점,11,7.10
...,...,...,...,...
190,탕후루,홀전문점,1,0.11
191,테이크아웃커피,홀전문점,1,0.11
192,한식-죽,홀전문점,1,0.11
193,한식뷔페,홀전문점,1,0.11


In [8]:
# 배달유형별 상권 개수 테이블
상권_분포 = (
    result.groupby(['상권','배달유형'])
      .size()
      .reset_index(name='가맹점수')
      .sort_values(['배달유형', '가맹점수'], ascending=[True, False])
)

# 배달유형별 비율 계산
상권_비율 = (
    상권_분포
    .groupby('배달유형')
    .apply(lambda x: x.assign(비율=round(x['가맹점수'] / x['가맹점수'].sum() * 100, 2)))
    .reset_index(drop=True)
)
상권_비율

,상권,배달유형,가맹점수,비율
0,왕십리,배달전문점,61,39.35
1,금남시장,배달전문점,26,16.77
2,마장동,배달전문점,21,13.55
3,성수,배달전문점,15,9.68
4,신금호,배달전문점,10,6.45
5,옥수,배달전문점,7,4.52
6,답십리,배달전문점,4,2.58
7,한양대,배달전문점,4,2.58
8,행당,배달전문점,4,2.58
9,뚝섬,배달전문점,1,0.65


# 배달 비율에 따른 매출금액

In [9]:
# 조사 제외
df_valid = df[df['배달매출금액 비율'] > 0].copy()

# 배달비율과 매출금액/건수의 상관계수
corr = df_valid[['배달매출금액 비율', '매출금액 구간', '매출건수 구간']].corr()
corr

,배달매출금액 비율,매출금액 구간,매출건수 구간
배달매출금액 비율,1.000000,0.611711,0.597753
매출금액 구간,0.611711,1.000000,0.757950
매출건수 구간,0.597753,0.757950,1.000000


In [10]:
# 배달비율 구간 나누기
bins = [0, 20, 40, 60, 80, 100]
labels = ['0~20%', '20~40%', '40~60%', '60~80%', '80~100%']
df_valid['배달비율구간'] = pd.cut(df_valid['배달매출금액 비율'], bins=bins, labels=labels, include_lowest=True)

# 각 구간별 평균 매출금액/건수
grouped = df_valid.groupby('배달비율구간')[['매출금액 구간', '매출건수 구간']].mean().round(2)
grouped

C:\Users\User\AppData\Local\Temp\ipykernel_76980\1665247341.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_valid.groupby('배달비율구간')[['매출금액 구간', '매출건수 구간']].mean().round(2)


,매출금액 구간,매출건수 구간
배달비율구간,,
0~20%,2.66,2.43
20~40%,2.94,2.69
40~60%,3.42,3.01
60~80%,3.97,3.60
80~100%,5.04,4.80


In [14]:
!pip install statsmodels


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\User\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [ ]:
import statsmodels.api as sm

# X: 배달비율, y: 매출금액 구간
X = sm.add_constant(df_valid['배달매출금액 비율'])
y = df_valid['매출금액 구간']

model = sm.OLS(y, X).fit()
model

ModuleNotFoundError: No module named 'statsmodels'

In [13]:
# 더미 변수화
df_valid = pd.get_dummies(df_valid, columns=['업종', '상권'], drop_first=True)

X = sm.add_constant(df_valid[['배달매출금액 비율'] + [col for col in df_valid.columns if col.startswith(('업종_', '상권_'))]])
y = df_valid['매출금액 구간']

model2 = sm.OLS(y, X).fit()
model2

NameError: name 'sm' is not defined